<a href="https://colab.research.google.com/github/guybanbo/IR-Query-Kings/blob/main/H3/KIZUR_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
!pip install requests
!pip install beautifulsoup4
!pip install firebase
!pip install ipython
!pip install langdetect
!pip install firebase
!pip install nltk
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=0f63657a0f92388b8ff2a8b36b8cd327cfaa4b74d9c9a2c54d27c3df2f54566a
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

crawler

In [ ]:

import requests
from firebase import firebase
from bs4 import BeautifulSoup
from collections import defaultdict
import json
import time
from urllib.parse import urljoin, urlparse,urlunparse


import requests
from urllib.parse import urlparse, urljoin,quote, unquote
import re
import nltk







#works with hebrew title
def links_enrichment(links):
    meta = []
    for link in links:
        try:
            # Create a session object to manage redirections
            time.sleep(1)
            session = requests.Session()
            session.max_redirects = 10

            # Fetch the page content
            response = session.get(link, timeout=10)
            response.raise_for_status()

            # Ensure proper encoding (UTF-8) for Hebrew text
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract meta description
            meta_description = soup.find('meta', {'name': 'description'})
            description = meta_description['content'] if meta_description else "תיאור אינו זמין"

            # Extract title
            title_tag = soup.find('title')
            title = title_tag.text if title_tag else "כותרת אינה זמינה"

            # Print the title for debugging
            print(f"Title: {title}")

            # Append metadata to the result
            meta.append({
                "link": link,
                "title": title.strip(),
                "description": description.strip()
            })
        except requests.Timeout:
            print(f"Timeout crawling {link}")
        except Exception as e:
            print(f"Error crawling {link}: {e}")
            meta.append({
                "link": link,
                "title": "כותרת אינה זמינה",
                "description": "תיאור אינו זמין"
            })
    return meta



# Function to delete a db table
def delete_node(node_path):
    delete_url = f"{url}/{node_path}.json"
    response = requests.delete(delete_url)
    if response.status_code == 200:
        print(f"Node '{node_path}' deleted successfully")
    else:
        print(f"Failed to delete node '{node_path}', Status code: {response.status_code}")






def decode_url(href):
    """Decode a URL that might be double-encoded."""
    try:
        # Decode the URL twice to handle double encoding
        decoded = unquote(unquote(href))
        return decoded
    except Exception as e:
        print(f"Error decoding URL: {href} - {e}")
        return href  # Return original if decoding fails


visited = set()

def crawl(url, max_depth, depth=0):
    if depth > max_depth:  # Stop if maximum depth is exceeded
        return

    if url in visited:  # Skip already visited URLs
        return

    visited.add(url)  # Mark the URL as visited
    print(f"Crawling: {url}")

    # Create a session object to manage redirections
    session = requests.Session()
    session.max_redirects = 10

    try:
        # Fetch the page content
        time.sleep(1)  # Respectful crawling: add a delay
        response = session.get(url, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Set the encoding explicitly for Hebrew support
        response.encoding = 'utf-8'

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract and process links
        for link in soup.find_all('a', href=True):
            href = link['href']


            # Decode the URL and re-encode it properly
            decoded_href = decode_url(href)
            encoded_href = quote(decoded_href, safe="/:&?=")  # Re-encode for proper URL usage

            # Resolve relative URLs into absolute URLs
            absolute_url = urljoin(url, encoded_href)

            # Validate domain to stay within the target site
            domain = urlparse(absolute_url).netloc
            if domain and (domain.endswith('.kizur.co.il') or domain == 'kizur.co.il'):
                if absolute_url not in visited:  # Avoid re-crawling
                    crawl(absolute_url, max_depth, depth + 1)

    except requests.Timeout:
        print(f"Timeout crawling {url}")
    except Exception as e:
        print(f"Error crawling {url}: {e}")

# Start crawling from the root URL
root_url = "http://www.kizur.co.il/home.php"
max_depth = 1
crawl(root_url, max_depth)
crawled_links =list(visited)
links = links_enrichment(crawled_links)


url = "https://query-kings-default-rtdb.europe-west1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(url,None)

for link in links:
  FBconn.post('/pagesl/',link)








export pages

In [ ]:
import requests
from firebase import firebase
import openpyxl
from openpyxl.utils import get_column_letter

# Initialize Firebase connection
url = "https://query-kings-default-rtdb.europe-west1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(url, None)

# Function to fetch data from Firebase
def fetch_data_from_firebase():
    # Get data from Firebase
    result = FBconn.get('/pages', None)
    if result:
        return result
    else:
        print("No data found in Firebase")
        return {}

# Function to create and save Excel file
def save_links_to_excel(links_data):
    # Create a new workbook and sheet
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Pages"

    # Set the header row
    ws['A1'] = "Link"
    ws['B1'] = "Title"

    # Write the data to the sheet
    row = 2  # Start from row 2 to leave space for headers
    for link, data in links_data.items():
        ws[f'A{row}'] = data.get('link', 'No Link')
        ws[f'B{row}'] = data.get('title', 'No Title')
        row += 1

    # Expand columns based on content length
    for col in ['A', 'B']:
        max_length = 0
        for cell in ws[col]:
            try:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
            except Exception:
                pass
        adjusted_width = max_length + 2  # Adjust width with a little padding
        ws.column_dimensions[get_column_letter(ws[col][0].column)].width = adjusted_width

    # Save the workbook to a file
    wb.save("pages.xlsx")
    print("Data successfully saved to 'pages.xlsx'")

# Fetch data from Firebase
links_data = fetch_data_from_firebase()

# Save the data to Excel
save_links_to_excel(links_data)


Data successfully saved to 'pages.xlsx'


query 1

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
from collections import defaultdict, Counter
import re
from firebase import firebase
import math


def get_content(link):
    try:
        time.sleep(1)
        session = requests.Session()
        session.max_redirects = 10
        response = session.get(link, timeout=10)
        response.raise_for_status()
        response.encoding = 'utf-8'
        return response.text
    except requests.Timeout:
        print(f"Timeout crawling {link}")
    except Exception as e:
        print(f"Error crawling {link}: {e}")
    return ""

def extract_acronym_definitions(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr', valign='top')
    pairs = []
    for row in rows:
        columns = row.find_all('td', class_='sr_results')
        if len(columns) == 2:
            acronym = columns[0].get_text(strip=True)
            definition = columns[1].get_text(strip=True)
            pairs.append((acronym, definition))
    return pairs
#the average number of words in the definitions of abbreviations
#for categories of organizations and institutions
def query1(links):
    total_words = 0
    total_definitions = 0

    all_data = []
    document_word_index = defaultdict(list)  # Dictionary to track words per document
    all_words = []  # List to track all words for frequency analysis

    for doc_id, data in links.items():
        link = data["link"]
        print(f"Processing: {link}")
        html_content = get_content(link)
        if not html_content:
            continue
        pairs = extract_acronym_definitions(html_content)

        # Extract the page number from the link
        page_match = re.search(r"page=(\d+)", link)
        page_number = page_match.group(1) if page_match else "unknown"

        for acronym, definition in pairs:
            all_data.append({
                "Link": link,
                "Acronym": acronym,
                "Definition": definition
            })
            total_words += len(definition.split())
            total_definitions += 1

            # Add words from the definition to the index for this page
            words = definition.split()
            document_word_index[page_number].extend(words)
            all_words.extend(words)  # Collect all words for frequency analysis

    # Calculate average words per definition
    avg_words = total_words / total_definitions if total_definitions > 0 else 0

    # Add average as a separate row
    all_data.append({
        "Link": "Average Across All Definitions",
        "Acronym": "",
        "Definition": f"Average Words: {avg_words:.2f}"
    })

    # Prepare document index data
    index_data = []
    for page_number, words in document_word_index.items():
        index_data.append({
            "Document": f"{page_number}",
            "Index": ",".join(sorted(set(words)))
        })

    # Get the 15 most common words
    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(15)
    common_words_data = [
        {"Word": word, "Count": count} for word, count in most_common_words
    ]

    # Build inverted index for the 15 most common words
    inverted_index_data = []
    for word, _ in most_common_words:
        pages_containing_word = [
            f"{page_number}" for page_number, words in document_word_index.items() if word in words
        ][:20]  # Limit to a maximum of 20 pages
        inverted_index_data.append({
            "Word": word,
            "Documents": ",".join(pages_containing_word)
        })

    # Calculate TF-IDF
    total_documents = len(document_word_index)
    tfidf_data = []
    for word, _ in most_common_words:
        idf = (
            math.log(total_documents / sum(1 for words in document_word_index.values() if word in words))
        )
        for page_number, words in document_word_index.items():
            tf = words.count(word)
            tfidf = tf * idf
            tfidf_data.append({
                "Word": word,
                "Page": int(page_number),
                "TF": tf,
                "IDF": idf,
                "TF-IDF": tfidf
            })

    # Convert to DataFrames
    df_acronyms = pd.DataFrame(all_data)
    df_index = pd.DataFrame(index_data)
    df_common_words = pd.DataFrame(common_words_data)
    df_inverted_index = pd.DataFrame(inverted_index_data)
    df_tfidf = pd.DataFrame(tfidf_data)
    # Group by "Word" and sort by "Page" within each group
    from itertools import groupby

# Sort the data first by "Word" then by "Page"
    tfidf_data.sort(key=lambda x: (x['Word'], x['Page']))
# Convert to DataFrame
    df_tfidf = pd.DataFrame(tfidf_data)


    # Export to Excel with styling
    filename = 'Query1.xlsx'
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df_acronyms.to_excel(writer, index=False, sheet_name='Acronyms')
        df_index.to_excel(writer, index=False, sheet_name='Index')
        df_common_words.to_excel(writer, index=False, sheet_name='Common Words')
        df_inverted_index.to_excel(writer, index=False, sheet_name='Inverted Index')
        df_tfidf.to_excel(writer, index=False, sheet_name='TF-IDF')
        workbook = writer.book

        # Style Acronyms sheet
        worksheet_acronyms = writer.sheets['Acronyms']
        header_font = Font(bold=True)
        for col in worksheet_acronyms.iter_cols(min_row=1, max_row=1):
            for cell in col:
                cell.font = header_font
                cell.alignment = Alignment(horizontal='center', vertical='center')

        for col in worksheet_acronyms.columns:
            max_length = 0
            col_letter = col[0].column_letter  # Get the column letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except Exception:
                    pass
            adjusted_width = max_length + 2
            worksheet_acronyms.column_dimensions[col_letter].width = adjusted_width


    print(f"Data exported to {filename} successfully.")

# Firebase connection and data fetching
url = "https://query-kings-default-rtdb.europe-west1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(url, None)
result = FBconn.get('/pages', None)
filtered_records = {
    doc_id: data for doc_id, data in result.items()
    if data["link"].startswith('http://www.kizur.co.il/search_group.php?group=10')
}
query1(filtered_records)


Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=11
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=3
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=2
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=9
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=7
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=10
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=8
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=4
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=6
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=1
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=5
Processing: http://www.kizur.co.il/search_group.php?group=10&sub_group=0&page=12
Processing: http://www.kizur.co.il/se

# Query 2


In [ ]:

import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
from collections import defaultdict, Counter
import re

def get_content(link):
    try:
        time.sleep(1)
        session = requests.Session()
        session.max_redirects = 10
        response = session.get(link, timeout=10)
        response.raise_for_status()
        response.encoding = 'utf-8'
        return response.text
    except requests.Timeout:
        print(f"Timeout crawling {link}")
    except Exception as e:
        print(f"Error crawling {link}: {e}")
    return ""

def extract_acronym_definitions(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr', valign='top')
    pairs = []
    for row in rows:
        columns = row.find_all('td', class_='sr_results')
        if len(columns) == 2:
            acronym = columns[0].get_text(strip=True)
            definition = columns[1].get_text(strip=True)
            pairs.append((acronym, definition))
    return pairs
#the most common words in defintions of acronyms in the category Economy
def query2(links):
    total_words = 0
    total_definitions = 0

    all_data = []
    document_word_index = defaultdict(set)  # Dictionary to track words per document
    all_words = []  # List to track all words for frequency analysis

    for doc_id, data in links.items():
        link = data["link"]
        print(f"Processing: {link}")
        html_content = get_content(link)
        if not html_content:
            continue
        pairs = extract_acronym_definitions(html_content)

        # Extract the page number from the link
        page_match = re.search(r"page=(\d+)", link)
        page_number = page_match.group(1) if page_match else "unknown"

        for acronym, definition in pairs:
            all_data.append({
                "Link": link,
                "Acronym": acronym,
                "Definition": definition
            })
            total_words += len(definition.split())
            total_definitions += 1

            # Add words from the definition to the index for this page
            words = definition.split()
            document_word_index[page_number].update(words)
            all_words.extend(words)  # Collect all words for frequency analysis

    # Prepare document index data
    index_data = []
    for page_number, words in document_word_index.items():
        index_data.append({
            "Document": f"{page_number}",
            "Index": ",".join(sorted(words))
        })

    # Get the 15 most common words
    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(15)
    common_words_data = [
        {"Word": word, "Count": count} for word, count in most_common_words
    ]

    # Build inverted index for the 15 most common words
    inverted_index_data = []
    for word, _ in most_common_words:
        pages_containing_word = [
            f"{page_number}" for page_number, words in document_word_index.items() if word in words
        ][:20]  # Limit to a maximum of 20 pages
        inverted_index_data.append({
            "Word": word,
            "Documents": ",".join(pages_containing_word)
        })

    # Convert to DataFrames
    df_acronyms = pd.DataFrame(all_data)
    df_index = pd.DataFrame(index_data)
    df_common_words = pd.DataFrame(common_words_data)
    df_inverted_index = pd.DataFrame(inverted_index_data)

    # Export to Excel with styling
    filename = 'Query 2.xlsx'
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df_acronyms.to_excel(writer, index=False, sheet_name='Acronyms')
        df_index.to_excel(writer, index=False, sheet_name='Index')
        df_common_words.to_excel(writer, index=False, sheet_name='Common Words')
        df_inverted_index.to_excel(writer, index=False, sheet_name='Inverted Index')

        workbook = writer.book

        # Style Acronyms sheet
        worksheet_acronyms = writer.sheets['Acronyms']
        header_font = Font(bold=True)
        for col in worksheet_acronyms.iter_cols(min_row=1, max_row=1):
            for cell in col:
                cell.font = header_font
                cell.alignment = Alignment(horizontal='center', vertical='center')

        for col in worksheet_acronyms.columns:
            max_length = 0
            col_letter = col[0].column_letter  # Get the column letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except Exception:
                    pass
            adjusted_width = max_length + 2
            worksheet_acronyms.column_dimensions[col_letter].width = adjusted_width

        # Style Index sheet
        worksheet_index = writer.sheets['Index']
        for col in worksheet_index.iter_cols(min_row=1, max_row=1):
            for cell in col:
                cell.font = header_font
                cell.alignment = Alignment(horizontal='center', vertical='center')

        for col in worksheet_index.columns:
            max_length = 0
            col_letter = col[0].column_letter  # Get the column letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except Exception:
                    pass
            adjusted_width = max_length + 2
            worksheet_index.column_dimensions[col_letter].width = adjusted_width

        # Style Common Words sheet
        worksheet_common_words = writer.sheets['Common Words']
        for col in worksheet_common_words.iter_cols(min_row=1, max_row=1):
            for cell in col:
                cell.font = header_font
                cell.alignment = Alignment(horizontal='center', vertical='center')

        for col in worksheet_common_words.columns:
            max_length = 0
            col_letter = col[0].column_letter  # Get the column letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except Exception:
                    pass
            adjusted_width = max_length + 2
            worksheet_common_words.column_dimensions[col_letter].width = adjusted_width

        # Style Inverted Index sheet
        worksheet_inverted_index = writer.sheets['Inverted Index']
        for col in worksheet_inverted_index.iter_cols(min_row=1, max_row=1):
            for cell in col:
                cell.font = header_font
                cell.alignment = Alignment(horizontal='center', vertical='center')

        for col in worksheet_inverted_index.columns:
            max_length = 0
            col_letter = col[0].column_letter  # Get the column letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except Exception:
                    pass
            adjusted_width = max_length + 2
            worksheet_inverted_index.column_dimensions[col_letter].width = adjusted_width

    print(f"Data exported to {filename} successfully.")


from firebase import firebase
url = "https://query-kings-default-rtdb.europe-west1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(url, None)
result = FBconn.get('/pages', None)
filtered_records = {
    doc_id: data for doc_id, data in result.items()
    if data["link"].startswith('http://www.kizur.co.il/search_group.php?group=14')
}
query2(filtered_records)


Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=3
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=7
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=10
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=8
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=4
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=9
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=5
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=2
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=11
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=6
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=1
Processing: http://www.kizur.co.il/search_group.php?group=14&sub_group=0&page=12
Processing: http://www.kizur.co.il/se

query 3


In [ ]:

import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
from collections import defaultdict, Counter
import re

def get_content(link):
    try:
        time.sleep(1)
        session = requests.Session()
        session.max_redirects = 10
        response = session.get(link, timeout=10)
        response.raise_for_status()
        response.encoding = 'utf-8'
        return response.text
    except requests.Timeout:
        print(f"Timeout crawling {link}")
    except Exception as e:
        print(f"Error crawling {link}: {e}")
    return ""

def extract_acronym_definitions_with_categories(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr', valign='top')
    pairs = set()  # Use set to ensure unique pairs

    for row in rows:
        columns = row.find_all('td', class_='sr_results')
        if len(columns) >= 3:  # Ensure there are at least three columns
            acronym = columns[0].get_text(strip=True).lower()
            definition = columns[1].get_text(strip=True).lower()

            # Extract categories
            category_links = columns[2].find_all('a', class_='group_word_link') if columns[2] else []
            categories = ", ".join(link.get_text(strip=True) for link in category_links)
            if not categories:
              categories = "אחר"  # Default to "אחר" if no categories found


            pairs.add((acronym, definition, categories))
    return list(pairs)


def query3(links):
    acronym_def_count = defaultdict(int)
    all_data = []
    document_word_index = defaultdict(set)
    all_words = []
    seen_pairs = set()

    for idx, data in enumerate(links, start=1):
        page_number = str(idx)
        link = data["link"]
        print(f"Processing: {link}")
        html_content = get_content(link)
        if not html_content:
            continue
        pairs = extract_acronym_definitions_with_categories(html_content)

        for acronym, definition, categories in pairs:
            if (acronym, definition, categories) not in seen_pairs:
                seen_pairs.add((acronym, definition, categories))
                all_data.append({
                    "Link": link,
                    "Acronym": acronym,
                    "Definition": definition,
                    "Category": categories,
                    "Page": page_number,
                })
                acronym_def_count[acronym] += 1
                words = definition.split()
                document_word_index[page_number].update(words)
                all_words.extend(words)

    # Calculate statistics
    total_acronyms = len(acronym_def_count)
    total_definitions = sum(acronym_def_count.values())
    avg_definitions_per_acronym = total_definitions / total_acronyms if total_acronyms > 0 else 0

    all_data.append({
        "Link": "Average Across All Acronyms",
        "Acronym": "",
        "Definition": f"Average Definitions: {avg_definitions_per_acronym:.2f}",
        "Category": "",
        "Page": "",
    })

    # Prepare document index and common word data
    index_data = []
    for page_number, words in document_word_index.items():
        index_data.append({
            "Document": f"{page_number}",
            "Index": ",".join(sorted(words)),
        })

    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(15)
    common_words_data = [{"Word": word, "Count": count} for word, count in most_common_words]

    inverted_index_data = []
    for word, _ in most_common_words:
        pages_containing_word = [
            f"{page_number}" for page_number, words in document_word_index.items() if word in words
        ][:20]
        inverted_index_data.append({
            "Word": word,
            "Documents": ",".join(pages_containing_word),
        })

    # Convert to DataFrames
    df_acronyms = pd.DataFrame(all_data)
    df_index = pd.DataFrame(index_data)
    df_common_words = pd.DataFrame(common_words_data)
    df_inverted_index = pd.DataFrame(inverted_index_data)

    # Export to Excel
    filename = 'Query 3.xlsx'
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        df_acronyms.to_excel(writer, index=False, sheet_name='Acronyms')
        df_index.to_excel(writer, index=False, sheet_name='Index')
        df_common_words.to_excel(writer, index=False, sheet_name='Common Words')
        df_inverted_index.to_excel(writer, index=False, sheet_name='Inverted Index')

        workbook = writer.book
        header_font = Font(bold=True)

        for sheet_name in ['Acronyms', 'Index', 'Common Words', 'Inverted Index']:
            worksheet = writer.sheets[sheet_name]
            for col in worksheet.iter_cols(min_row=1, max_row=1):
                for cell in col:
                    cell.font = header_font
                    cell.alignment = Alignment(horizontal='center', vertical='center')

            for col in worksheet.columns:
                max_length = 0
                col_letter = col[0].column_letter
                for cell in col:
                    try:
                        if cell.value:
                            max_length = max(max_length, len(str(cell.value)))
                    except Exception:
                        pass
                adjusted_width = max_length + 2
                worksheet.column_dimensions[col_letter].width = adjusted_width

    print(f"Data exported to {filename} successfully.")

from firebase import firebase
url = "https://query-kings-default-rtdb.europe-west1.firebasedatabase.app/"
FBconn = firebase.FirebaseApplication(url, None)
result = FBconn.get('/pages', None)
# Run the updated query function
filtered_records = [
    data for doc_id, data in result.items()
    if data["link"].startswith('http://www.kizur.co.il/search_word')
]

query3(filtered_records)



Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%91%D7%90%D7%A2%D7%94%22%D7%97&m=23
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%A0%D7%A1&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%91%D7%90%22%D7%9E&m=23
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%A0%D7%9C%D7%90&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%96%D7%94%D7%90&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%91%D7%97&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%90%D7%A7&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%9E%D7%9E%D7%93&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%96%D7%95%D7%AA&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%99%D7%95&m=3
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%A4%D7%9E%22%D7%90&m=23
Processing: http://www.kizur.co.il/search_word.php?abbr=%D7%A4%D6%BC%D6%B8%D7%9E%D6%B8%22%D7%96&m=23
Proces

page rank

In [ ]:
def calculate_new_pagerank(current_ranks, links, damping_factor=0.85):
    num_pages = len(current_ranks)
    new_ranks = {page: (1 - damping_factor) / num_pages for page in current_ranks}

    # Calculate contributions from incoming links
    for page in current_ranks:
        for target in links.get(page, []):  # Outgoing links from page
            num_outgoing = len(links[page])
            if num_outgoing > 0:
                new_ranks[target] += damping_factor * (current_ranks[page] / num_outgoing)

    return new_ranks


def print_ranks(ranks, iteration):
    print(f"\nPageRank values after iteration {iteration}:")
    print("-" * 35)
    print("Page  |  PageRank Value")
    print("-" * 35)
    for page, rank in sorted(ranks.items()):
        print(f"  {page}   |     {rank:.5f}")
    print("-" * 35)

urls=["http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%92%D7%97&m=3", "http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%92%D7%97%D7%A9&m=3","http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%92%D7%A9&m=3"
,"http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%92%D7%99&m=3","http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%91%D7%90&m=3", "http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%93&m=3"
"http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%91%D7%92&m=3","http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%90&m=3","http://www.kizur.co.il/search_word.php?abbr=%D7%90&m=3"
,"http://www.kizur.co.il/search_word.php?abbr=%D7%90%D7%90%D7%91&m=3"]


# Assign letters to URLs
url_to_letter = {url: chr(65 + i) for i, url in enumerate(urls)}

# Web structure (links between pages)
links = {
    'A': ['B', 'D'],
    'B': ['D'],
    'C': ['F', 'G'],
    'D': ['E'],
    'E': ['G'],
    'F': ['G'],
    'G': ['H', 'I'],
    'H': ['I'],
    'I': ['H', 'J'],
    'J': ['H', 'I'],
}

# Initialize PageRank values (1/number_of_pages for each page)
pages = list(links.keys())
num_pages = len(pages)
current_ranks = {page: 1 / num_pages for page in pages}

# Print initial values
print("\nInitial PageRank Values:")
print_ranks(current_ranks, 0)

# Perform iterations
iterations = 5
for i in range(1, iterations + 1):
    current_ranks = calculate_new_pagerank(current_ranks, links)
    print_ranks(current_ranks, i)

# Find highest PageRank after all iterations
highest_page = max(current_ranks.items(), key=lambda x: x[1])
print(f"\nHighest PageRank after {iterations} iterations:")
print(f"Page {highest_page[0]} with PageRank value of {highest_page[1]:.5f}")



Initial PageRank Values:

PageRank values after iteration 0:
-----------------------------------
Page  |  PageRank Value
-----------------------------------
  A   |     0.10000
  B   |     0.10000
  C   |     0.10000
  D   |     0.10000
  E   |     0.10000
  F   |     0.10000
  G   |     0.10000
  H   |     0.10000
  I   |     0.10000
  J   |     0.10000
-----------------------------------

PageRank values after iteration 1:
-----------------------------------
Page  |  PageRank Value
-----------------------------------
  A   |     0.01500
  B   |     0.05750
  C   |     0.01500
  D   |     0.14250
  E   |     0.10000
  F   |     0.05750
  G   |     0.22750
  H   |     0.14250
  I   |     0.18500
  J   |     0.05750
-----------------------------------

PageRank values after iteration 2:
-----------------------------------
Page  |  PageRank Value
-----------------------------------
  A   |     0.01500
  B   |     0.02138
  C   |     0.01500
  D   |     0.07025
  E   |     0.13613
  F   